In [2]:
import numpy as np
# ALGORITHM: RSVD — BASIC RANDOMIZED SVD
def randomized_svd(A, k, p):
    """
    A = QQ'A = QB = QUhatDV' = UDV'
    parameters:
        A - m*n matrix
        k - target rank
        p - over-sampling parameter
        
    Outputs:
        matrices U,D,V in an approximate rank-(k+p) svd of A
    """
    m, n = A.shape
    # form an n*(k + p) Gaussian random matrix G
    G = np.random.rand(n, k+p)
    # form the sample matrix Y = AG.
    Y = A@G
    # orthonormalize the columns of the sample matrix Q = orth(Y)
    Q = np.linalg.qr(Y, mode = 'reduced')[0]
    # form the (k + p) * n matrix B = Q'*A
    B = Q.T@A
    # form the svd of the small matrix B
    Uhat, D, V = np.linalg.svd(B, full_matrices=False)
    # drop the last p components
    Uhat = Uhat[:,:k]; D = D[:k]; V = V[:,:k];
    # form U = Q * Uhat
    U = Q@Uhat
    D = np.diag(D)
    return U, D, V